In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
from math import pi, sin, cos
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()
_base_ = '../../data/2013_vast_challenge/mc3_netflow/nf/'
df_orig = pl.concat([pl.read_csv(_base_ + 'nf-chunk1.csv'), pl.read_csv(_base_ + 'nf-chunk2.csv'), pl.read_csv(_base_ + 'nf-chunk3.csv')])
df_orig = df_orig.rename({'TimeSeconds':'secs',                  'parsedDate':'timestamp',                'dateTimeStr':'timestamp_str',
                          'ipLayerProtocol':'pro_str',           'ipLayerProtocolCode':'pro',             'firstSeenSrcIp':'sip',
                          'firstSeenDestIp':'dip',               'firstSeenSrcPort':'spt',                'firstSeenDestPort':'dpt',
                          'moreFragments':'mfrag',               'contFragments':'cfrag',                 'durationSeconds':'dur',
                          'firstSeenSrcPayloadBytes':'soct_pay', 'firstSeenDestPayloadBytes':'doct_pay',  'firstSeenSrcTotalBytes':'soct',
                          'firstSeenDestTotalBytes':'doct',      'firstSeenSrcPacketCount':'spkt',        'firstSeenDestPacketCount':'dpkt',
                          'recordForceOut':'out'})
df_orig = rt.columnsAreTimestamps(df_orig, 'timestamp')
#
# original implementation of edge bundling ... which connected all edge nodes to the first tier (wasting a lot of graph edges)
# ... following config requires 15.6s for path calcs... 2nd run = 16.0s ... 3rd run = 15.7s // 7900x @ 96gb ram
# ... with improvement (not connecting edge nodes to the first tier), the path calc time drops to 0.16s
#
_sips_ = set(['172.10.0.6','172.10.0.3','172.10.0.4','10.199.25.0.2','10.0.0.13','10.0.0.7']) # top elements in the histogram
_dips_ = set(['172.0.0.1']) # top element(s) in the histogram
df      = df_orig.filter(pl.col('sip').is_in(_sips_) | pl.col('dip').is_in(_dips_))
print(f'{len(df)=}')
rt.tile([rt.histogram(df_orig,bin_by='sip',count_by='dip'),
         rt.histogram(df_orig,bin_by='dip',count_by='sip'),
         rt.histogram(df,     bin_by='sip',count_by='dip'),
         rt.histogram(df,     bin_by='dip',count_by='sip')])

In [ ]:
cd = rt.chordDiagram(df, [('sip','dip')], link_style='bundled', skeleton_algorithm='hdbscan')
cd

In [ ]:
cd = rt.chordDiagram(df, [('sip','dip')], link_style='bundled', skeleton_algorithm='hexagonal')
cd